# Instructions for Setting up Valves, Tubing and Reservoirs
## Setting up the Valves
+ We will use two kinds of Valves [3:2 Valves and 2:2 Valves](instructions_for_assembling_valve_controller.ipynb).
+ 3:2 Valves will be used for the Cell, Oil and Bead Bypass Valves.
+ We have defined two classes for 3:2 and 2:2 Valves.
    + We also have a derived class called Time Point valves from 2:2 Valves.
+ We will use these Class Objects to Identify each valve in our device.
+ Execute the cells below

In [1]:
# coding : utf8
%matplotlib inline
import os
import sys
sys.path.append(os.getcwd());
import ivPID.PID as PID_controller

print(os.getcwd())
os.chdir(".\config")

#Adding required directories to sys.path. Make sure you are in the Directory with the Git Repo when you start Jupyter Notebook
sys.path.append(os.getcwd());
sys.path.append(os.getcwd()+"\\DLL64");

import time
import Elveflow64
os.chdir("..")
from ctypes import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import serial
import threading
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.schedulers import SchedulerNotRunningError
from datetime import datetime,timedelta
import seaborn as sns
import pygame

C:\Users\kanis\Documents\ChronoSeq
pygame 2.5.2 (SDL 2.28.3, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

### Note on Ipython Magics
+ We will store device specific configuration in the config folder using [Ipython Magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html#)
+ We modified the ```%%writefile``` magic as follows.
+ Execute the cell below

In [5]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

In [4]:
#Initializing Instrument
%run -i config/instrument_initialization.py

Pressure Controller Connection Started
Pressure Controller ID: 0
True
{'baudrate': 9600, 'bytesize': 8, 'parity': 'N', 'stopbits': 1, 'xonxoff': False, 'dsrdtr': False, 'rtscts': False, 'timeout': 1, 'write_timeout': 0, 'inter_byte_timeout': None}
True
{'baudrate': 9600, 'bytesize': 8, 'parity': 'N', 'stopbits': 1, 'xonxoff': False, 'dsrdtr': False, 'rtscts': False, 'timeout': 1, 'write_timeout': 0, 'inter_byte_timeout': None}
True
{'baudrate': 9600, 'bytesize': 8, 'parity': 'N', 'stopbits': 1, 'xonxoff': False, 'dsrdtr': False, 'rtscts': False, 'timeout': 1, 'write_timeout': 0, 'inter_byte_timeout': None}
COM12
{'baudrate': 9600, 'bytesize': 8, 'parity': 'N', 'stopbits': 1, 'xonxoff': False, 'dsrdtr': False, 'rtscts': False, 'timeout': 1, 'write_timeout': 0, 'inter_byte_timeout': None}
True
Cell Channel Flow Meter ID: 3000
error 0 :
Oil Channel Flow Meter ID: 3001
error 0 :
Cell Channel Density:
1.2680259943008423
Cell Channel Flow rate:
-1103.77129063566
Oil Channel Density:
2.139642

In [5]:
#Loading functions for setting Valve Controllers
%run -i config/setValveControllers.py

In [6]:
safety_time=5.0

class valve2_2:
    def __init__(self,valve_index,valve_controller_id,type_of_valve):
        """
        NC is a Normally Closed Valve: you need to turn it on to open it
        NO is a Normally Open Valve: you need to turn it on to close it
        2_2 valve means there are two bidirectional ports in the valve.
        A 3_2 valve can be converted to a 2_2 NO or 2_2 NC Valve by blocking the corresponding port on the valve
        """
        assert type_of_valve in ("NC","NO")
        assert type(valve_controller_id) is c_int32 
        self.type=type_of_valve
        self.index=valve_index
        self.controller=valve_controller_id
        if self.controller is valve_controller1:
            self.state_vector=valveController1_state
        elif self.controller is valve_controller2:
            self.state_vector=valveController2_state
        else:
            self.state_vector=valveController3_state
        
    def OPEN(self):
        if self.type is "NC":
            (self.state_vector)[self.index]=1
        else:
            (self.state_vector)[self.index]=0
        
    def CLOSE(self):
        if self.type is "NO":
            (self.state_vector)[self.index]=1
        else:
            (self.state_vector)[self.index]=0
               
class valve3_2:
    def __init__(self,valve_index,valve_controller_id,NC_Port,NO_Port,flip=False):
        """
        A 3_2 valve means there are 3 bidirectional ports on the valve.
        A 3_2 valve can be converted to a 2_2 NO or 2_2 NC Valve by blocking the corresponding port on the valve
        Look for the Ports marked NO and NC on the valve if you want to do this.
        """
        assert type(valve_controller_id) is c_int32 
        assert type(NC_Port) is str and type(NO_Port) is str
        self.NC_Port=NC_Port
        self.NO_Port=NO_Port
        self.index=valve_index
        self.controller=valve_controller_id
        self.flip=flip
        if self.controller is valve_controller1:
            self.state_vector=valveController1_state
        elif self.controller is valve_controller2:
            self.state_vector=valveController2_state
        else:
            self.state_vector=valveController3_state
        
    def ON(self):
        if self.flip is False:
            (self.state_vector)[self.index]=1
        else:
            (self.state_vector)[self.index]=0
        
    def OFF(self):
        if self.flip is False:
            (self.state_vector)[self.index]=0
        else:
            (self.state_vector)[self.index]=1
            
            
class time_point_valve(valve2_2):
    
    all_time_point_valves={}
    
    def __init__(self,valve_index,valve_controller_id,type_of_valve,time_point):
        """
        A time point valve is essentially a 2_2 valve but the length of the tubing from the reservoir all the way to the valve 
        matters. This is because the length determines the amount of time it will take at 2000mbar to reach the manifold when using
        a chip which is ~105um in height. These values need to be empirically determined depending on the length of tubing used. 
        """
        super().__init__(valve_index,valve_controller_id,type_of_valve)
        self.time_point=time_point
        time_point_valve.all_time_point_valves[time_point]=self
        
    def get_time_point(self):
        return self.time_point

### Instantiating Valve Objects

#### 2:2 Valve Example
+ For a valve2_2 Object the input values required are as follows ```valve2_2(<Position on Valve Controller-1>,<valve_controller1,valve_controller2, or valve_controller3>,<Type of Valve: Normally Closed (NC) or Normally Open (NO)>)```
+ Lets connect our first 2_2 Valve to see how this works.
+ We will connect our ```bead_lysis_buffer_flush_valve```
+ Put a SMC 2:2 Valve in the position shown.
    + Wrap Labelling Tape around the base to prevent it from moving.<br><img src="img/IMG_4462.jpg" height="50%" width="50%">
+ Label the Valve: Lysis Buffer.<br><img src="img/IMG_4463.jpg" height="50%" width="50%">
+ Connect it to the First Position on Valve Controller 2. <br><img src="img/IMG_4464.jpg" height="50%" width="50%">
+ Route the cable through the middle of the Valve Holder as shown. <br><img src="img/IMG_4466.jpg" height="50%" width="50%">
+ This is how we will instantiate our object: ```bead_lysis_buffer_flush_valve= valve2_2(0,valve_controller2,"NC") ```
    + Since we plugged into Position 1 for the Second Valve Controller, we will input the first value as ```0```.
    + Since we plugged this valve into the second Valve Controller, we will use the value ```valve_controller2```.
    + Lastly since this valve is Normally Closed, we will use the value ```"NC"```.
+ Execute the cell below to instantiate the Valve Object.
    + This will also create the ```config/valveObjects.py``` file.

In [7]:
%%write_and_run config/valveObjects.py
bead_lysis_buffer_flush_valve= valve2_2(0,valve_controller2,"NC") 

+ You are free to choose a different Valve Controller or Position on that Valve Controller, just make you input the correct values for the Valve Object.

#### 3:2 Valve Example
+ For a valve3_2 Object the input values required as as follows: ```valve3_2(<Position on Valve Controller-1>,<valve_controller1,valve_controller2, or valve_controller3>,<Text String for where the NC Port Connects to>,<Text String for where the NO Port Connects to>,<When Flip is True, the liquid will bypass the Microfluidic Chip when the Valve is ON >)```
+ Lets connect our first 3_2 Valve to see how this works.
+ We will connect our ```bead_bypass_valve```
+ Put a SMC 3:2 Valve in the position shown.
    + Wrap Labelling Tape around the base to prevent it from moving.
    + Label the Valve: Bead Bypass.<br><img src="img/IMG_4468.jpg" height="50%" width="50%">
+ Connect it to the Second Position on Valve Controller 2. <br><img src="img/IMG_4469.jpg" height="50%" width="50%">
+ This is how we will intantiate our object: ```bead_bypass_valve= valve3_2(1,valve_controller2,"Chip","XYZ Robot",True)```
    + Since we plugged into Position 2 for the Second Valve Controller, we will input the first value as ```1```.
    + Since we plugged this valve into the second Valve Controller, we will use the value ```valve_controller2```.
    + We will connect the NC Port of the 3:2 Valve to our Microfluidic Chip so we will specify this value as ```"Chip."```
    + We will connect the NO Port of the 3:2 Valve to our XYZ Robot so we will specify this value as ```"XYZ Robot"```
    + Lastly we want the liquid to flow to the XYZ Robot when the Bypass is ON, so we have set the Flip value to ```True```.
+ Execute the cell below to intantiate the Valve Object.
    + This will also update the ```config/valveObjects.py``` file.

In [8]:
%%write_and_run -a config/valveObjects.py
bead_bypass_valve= valve3_2(1,valve_controller2,"Chip","XYZ Robot",True)

+ You are free to choose a different Valve Controller or Position on that Valve Controller, just make you input the correct values for the Valve Object.

#### Time Point Valves
+ These valves will control the flow from Reservoirs labelled 1 through 12 for injecting Time-Tagged Beads into our Microfluidic Device.
+ Each Time point valve has a corresponding position with the same number label in Ice Box 2 where unused beads are deposited.
+ Each Time point valve controls the Bead Reservoir with the same number label.
+ For a time_point_valve Object the input values required as as follows: ```time_point_valve(<Position on Valve Controller-1>,<valve_controller1,valve_controller2, or valve_controller3>,<Type of Valve: Normally Closed (NC) or Normally Open (NO)>,<Number Label for Reservoir/Ice Box/Time Point Valve>)``` 
+ You can remove the Top part of the Valve Holder to make it easier to attach the Valves.<br><img src="img/IMG_4470.jpg" height="50%" width="50%">
+ We will now Connect and Label our Time point Valves as shown.
    + Make sure the orientation of the Valves matches that shown in the images below.
    + This orientation with the power cable pointing upwards helps minimize bead sedimentation inside the valve.
    + Label each valve from 1 through 12.

|Back View|Side View|Front View|
|-|-|-|
|<img src="img/IMG_4472.jpg">|<img src="img/IMG_4473.jpg">|<img src="img/IMG_4474.jpg">|

+ You should stack the Valve Controllers on top of each other and move the cables so they are behind the Taller Monitor Riser.<br><img src="img/IMG_4475.jpg" height="50%" width="50%">
+ Then put the excess length of cables on the rack above your bench secured with tape to reduce clutter.<br><img src="img/IMG_4500.jpg" height="50%" width="50%">
+ Execute the cell below to intantiate the Valve Objects.
    + Populate the List below for all 12 Time Point Valves.
    + This will also update the ```config/valveObjects.py``` file.

In [9]:
%%write_and_run -a config/valveObjects.py
bead_time_points_valve_list=[] 
#Set the values for Position on Valve Controller, and Valve Controller
bead_time_points_valve_list.append(time_point_valve(2,valve_controller2,"NC",1)) #First Time point valve
bead_time_points_valve_list.append(time_point_valve(3,valve_controller2,"NC",2)) #Second Time point valve 
bead_time_points_valve_list.append(time_point_valve(4,valve_controller2,"NC",3)) #Third Time point valve
bead_time_points_valve_list.append(time_point_valve(5,valve_controller2,"NC",4)) #Fourth Time point valve
bead_time_points_valve_list.append(time_point_valve(6,valve_controller2,"NC",5)) #Fifth Time point valve
bead_time_points_valve_list.append(time_point_valve(6,valve_controller3,"NC",6)) #Sixth Time point valve
bead_time_points_valve_list.append(time_point_valve(0,valve_controller3,"NC",7)) #Seventh Time point valve
bead_time_points_valve_list.append(time_point_valve(1,valve_controller3,"NC",8)) #Eighth Time point valve
bead_time_points_valve_list.append(time_point_valve(2,valve_controller3,"NC",9)) #Ninth Time point valve
bead_time_points_valve_list.append(time_point_valve(3,valve_controller3,"NC",10)) #Tenth Time point valve
bead_time_points_valve_list.append(time_point_valve(4,valve_controller3,"NC",11)) #Eleventh Time point valve
bead_time_points_valve_list.append(time_point_valve(5,valve_controller3,"NC",12)) #Twelveth Time point valve

#### Cell and Oil Bypass Valves
+ Place a 3:2 Valve in the Orientation shown in the first Valve Holder.
    + Label this valve: Oil Bypass <br><img src="img/IMG_4490.jpg" height="50%" width="50%">
+ Route the cable for this valve through the middle of the valve holder as shown.<br><img src="img/IMG_4491.jpg" height="50%" width="50%">
+ Place another 3:2 Valve in the Orientation shown.
    + Label this valve: Cell Bypass
    + Route the cable for this valve through the middle of the valve holder.<br><img src="img/IMG_4493.jpg" height="50%" width="50%">
+ Connect these valves to the Valve Controller 1. <br><img src="img/IMG_4495.jpg" height="50%" width="50%">
+ Execute the cell below to intantiate the Valve Objects.
    + Populate the List below for both Bypass Valves.
    + This will also update the ```config/valveObjects.py``` file.

In [10]:
%%write_and_run -a config/valveObjects.py
oil_bypass_valve=valve3_2(0,valve_controller1,"Chip","Reusable Oil",True)
cell_bypass_valve= valve3_2(1,valve_controller1,"Chip","Bleach Bottle",True)

#### Gas Vent and Cell Reservoir Valves
+ Place a 2:2 Valve in the Orientation shown.
    + Label this valve: Gas Vent. This will be our ```cell_gas_vent_valve```.
    + Route the cable for this valve through the middle of the valve holder.<br><img src="img/IMG_4496.jpg" height="50%" width="50%">
+ Place four 2:2 Valves in the Valve Holder as shown.
    + Label the last valve: Cell Media. This will be our ```cell_media_flush_valve```.
    + Label the first valve: Cell 1. This will be our ```cell_sample_flush_valve1```.
    + Label the second valve: Cell 2. This will be our ```cell_sample_flush_valve2```.
    + Label the third valve: Cell 3. This will be our ```cell_sample_flush_valve3```.
    
|Side View through the Sterlite Container|Top View|
|-|-|
|<img src="img/IMG_4497.jpg">|<img src="img/IMG_4498.jpg">|

+ Connect these valves to Valve Controller 1.<br><img src="img/IMG_4499.jpg" height="50%" width="50%">
+ Execute the cell below to intantiate the Valve Objects.
    + Populate the List below for all five Valves.
    + This will also update the ```config/valveObjects.py``` file.

In [11]:
%%write_and_run -a config/valveObjects.py
cell_gas_vent_valve=valve2_2(2,valve_controller1,"NC")
cell_media_flush_valve=valve2_2(3,valve_controller1,"NC")
cell_sample_flush_valve1=valve2_2(4,valve_controller1,"NC")
cell_sample_flush_valve2=valve2_2(5,valve_controller1,"NC")
cell_sample_flush_valve3=valve2_2(6,valve_controller1,"NC")

### Testing the Valves
+ Now we will Turn the Valves ON and then OFF one by one.
+ Touch the Valves in the order below to feel the valve clicking ON and OFF.
+ Fix any issues detected before moving forward.

In [12]:
#Defining Function to Test the Valves. Valves are turn on for 10 seconds, then turned off. 
#After 10 seconds the next valve gets test.
def test_valve(valve):
    print("Testing Valve:")
    if type(valve) is valve3_2 and valve.flip is True:
        valve.OFF()
        update_valve_states()
        time.sleep(10)
        reset_valve_states()
        time.sleep(10)
    elif type(valve) is valve2_2 or type(valve) is time_point_valve:
        valve.OPEN()
        update_valve_states()
        time.sleep(10)
        reset_valve_states()
        time.sleep(10)

#Making List of all Valves to test        
valves_to_test=[]

#Adding all Bead Time Point Valves to the List First
valves_to_test=valves_to_test+bead_time_points_valve_list

#After this remaining valves will be tested.
valves_to_test.append(bead_bypass_valve)
valves_to_test.append(bead_lysis_buffer_flush_valve)
valves_to_test.append(oil_bypass_valve)
valves_to_test.append(cell_bypass_valve)
valves_to_test.append(cell_gas_vent_valve)
valves_to_test.append(cell_media_flush_valve)
valves_to_test.append(cell_sample_flush_valve1)
valves_to_test.append(cell_sample_flush_valve2)
valves_to_test.append(cell_sample_flush_valve3)

for valve in valves_to_test:
    test_valve(valve)

Testing Valve:
Valve Controller 1 State successfully changed!
0 0 0 0 0 0 0 0 
Valve Controller 2 State successfully changed!
0 0 1 0 0 0 0 0 
Valve Controller 3 State successfully changed!
0 0 0 0 0 0 0 0 
Valve Controller 1 State successfully changed!
0 0 0 0 0 0 0 0 
Valve Controller 2 State successfully changed!
0 0 0 0 0 0 0 0 
Valve Controller 3 State successfully changed!
0 0 0 0 0 0 0 0 
Testing Valve:
Valve Controller 1 State successfully changed!
0 0 0 0 0 0 0 0 
Valve Controller 2 State successfully changed!
0 0 0 1 0 0 0 0 
Valve Controller 3 State successfully changed!
0 0 0 0 0 0 0 0 
Valve Controller 1 State successfully changed!
0 0 0 0 0 0 0 0 
Valve Controller 2 State successfully changed!
0 0 0 0 0 0 0 0 
Valve Controller 3 State successfully changed!
0 0 0 0 0 0 0 0 
Testing Valve:
Valve Controller 1 State successfully changed!
0 0 0 0 0 0 0 0 
Valve Controller 2 State successfully changed!
0 0 0 0 1 0 0 0 
Valve Controller 3 State successfully changed!
0 0 0 0 0 0 

+ Execute the cell below to shutdown the Device.

In [ ]:
#Device Shutdown.
%run -i config/deviceShutdown.py

## Setting up the Manifolds and Reservoirs
+ You will need the following items to get started.
    + [2 Units 1/16" ID 0.5mm ID PTFE tubing {CAT NO. JR-T-4183-M25}](https://www.vici.com/profiles/prof_ps.php) 
        + Email precision@vici.com for a Quote. Tell them your credit card details for payment.
    + [Stanley Razor Blade](https://www.grainger.com/product/STANLEY-Razor-Blade-1-1-2-in-Bladde-4A807) for cutting the PTFE Tubing.
    + [3 Units 100ft Soft Tygon PVC Plastic Tubing for Air&Water 3/32" ID, 7/32" OD](https://www.mcmaster.com/6516T52/)
    + [5 Units Plastic Quick-Turn Tube Coupling Tee Socket-to-Socket Connector, White Nylon Pack of 10](https://www.mcmaster.com/51525K443/)
    + Scissors for Cutting the Tygon Tubing.
    + [Metric Ruler](https://www.amazon.com/gp/product/B004E3NK92/)
    + [Metric Tape Measure](https://www.amazon.com/gp/product/B072KC4JF5/)
    + [2 Packs Bel-Art Scienceware 50ml Tube Holders](https://www.fishersci.com/shop/products/bel-art-scienceware-conical-tube-holder/11999001)
    + Permanent Marker
+ Order these items from Idex Health and Science:
    + [100 Units P-851 Luer Lock Male 3/32in Barb](https://www.idex-hs.com/store/product-detail/barbed_to_male_luer_lock_adapter_for_use_with_soft_walled_tubing_polypropylene_for_use_with_3_32_2_4_mm_id_tubing_each/p-851)
    + [1 Unit P-154 Manifold Assy 5 Port 1/16in](https://www.idex-hs.com/store/product-detail/peek_manifold_assembly_5_port_for_1_16_od/p-154)
    + [2 Units P-702-01 Union Body PEEK .020 thru hole 1/16](https://www.idex-hs.com/store/product-detail/union_body_peek_020_thru_hole_for_1_16_od_/p-702-01)
    + [30 Units D-647 Swivel Barb Ftg, 3/32in](https://www.idex-hs.com/store/product-detail/swivel_barb_adapter_for_use_with_soft_walled_tubing_polypropylene_each_for_3_32_2_4_mm_id_tubing/d-647)
    + [30 Units P-309 Flange Type Ftg Delrin Plug: Flat Bottom](https://www.idex-hs.com/store/product-detail/plug_delrin_1_4_28_flat_bottom/p-309)
    + [11 Units P-200X Flangeless Ferrule, Flat Bottom ETFE 1/16 Blue/10pk](https://www.idex-hs.com/store/product-detail/flangeless_ferrule_tefzel_etfe_1_4_28_flat_bottom_for_1_16_od_blue_10_pack/p-200x)
    + [4 Units P-632 Tee Assembly Tefzel 1/16 .020 thru hole](https://www.idex-hs.com/store/product-detail/tee_assembly_tefzel_etfe_for_1_16_od_020_thru_hole/p-632)
    + [25 Units P-206 Flangeless Male Nut 1/16 Blue](https://www.idex-hs.com/store/product-detail/flangeless_male_nut_delrin_1_4_28_flat_bottom_for_1_16_od_blue/p-206)
    + [25 Units P-203 Flangeless Male Nut 1/16in White](https://www.idex-hs.com/store/product-detail/flangeless_male_nut_delrin_1_4_28_flat_bottom_for_1_16_od_white/p-203)
    + [25 Units P-202 Flangeless Male Nut 1/16in Red](https://www.idex-hs.com/store/product-detail/flangeless_male_nut_delrin_1_4_28_flat_bottom_1_16_od_red/p-202)
    + [25 Units P-205 Flangeless Male Nut 1/16in Green](https://www.idex-hs.com/store/product-detail/flangeless_male_nut_delrin_1_4_28_flat_bottom_for_1_16_od_green/p-205)
    + [15 Units P-204 Flangeless Male Nut 1/16 Cream](https://www.idex-hs.com/store/product-detail/flangeless_male_nut_delrin_1_4_28_flat_bottom_for_1_16_od_cream/p-204)
    + [2 Units PEEK Manifold Assembly 9 Port, for 1/16" OD](https://www.idex-hs.com/store/product-detail/peek_manifold_assembly_9_port_for_1_16_od/p-191)

### Assembling the Bead Channel Tubing and Manifolds 
+ Cut a 10cm long piece of PTFE Tubing using the Stanley Razor Blade.
+ We need to use a Male Nut, and Flat Bottom Ferrule to attach this this PTFE tubing to our Valve.<br><img src="img/IMG_4514.jpg" height="50%" width="50%">
+ To attach our PTFE Tubing to our Lysis buffer flush valve we need to:
    + Constrict the end of our PTFE Tubing and prevent movement of the ferrule.
    + Make sure the cut plane of the PTFE Tubing and the Flat Bottom of the ferrule are in the same plane.
    + Any of the manifold fittings with a flat bottom, namely the 2 Port Union, Tee, 5 port manifold or 9 Port manifold can be used to make this happen.
        + Hold the Tubing in place and push it into the Manifold as you screw the Nut and Ferrule into the Manifold.
        + Hand tighten all the way till its not possible to screw in further.
    + Attach the PTFE Tubing to the NO Port of the Lysis Buffer Flush Valve.
    + Hand tighten all the way till its not possible to screw in further, to make sure there are no leaks.
    + Then attach a Tee to the other end of the PTFE Tubing.
    + Hand tighten all the way till its not possible to screw in further, to make sure there are no leaks.
    + Execute the cell below to watch a video on how to do this and attach the Tubing to your valve.<br><img src="img/IMG_4523.jpg" height="50%" width="50%">

In [3]:
%%HTML
<iframe width="800" height="600" src="https://www.youtube.com/embed/C1m8psZ4egE?si=58I6bOZN2-_HJJSg" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

+ For all Valves and Manifold connections: **Its important to make sure the ferrule and cut plane of the PTFE Tubing are in the same plane.**
+ For all connections unless mentioned otherwise: **Hand tighten all the way, till its not possible to screw in further. This is to make sure there are no leaks.**
+ Cut two 20cm Pieces of PTFE Tubing.<br><img src="img/IMG_4524.jpg" height="50%" width="50%">
+ Connect them to the tee joined to the NO Port of the Lysis Buffer Flush Valve. We want to reiterate:
    + Follow the steps described above for checking if the PTFE Tubing cut plane and Ferrule are in the same plane.
    + Hand tighten all the way till its not possible to screw in further, to make sure there are no leaks.<br><img src="img/IMG_4525.jpg" height="50%" width="50%">
+ Use labelling tape to secure the Tee in place as shown.<br><img src="img/IMG_4527.jpg" height="50%" width="50%">
+ Attach two Nuts and Ferrules to the other end of the PTFE Tubing on the Tees.<br><img src="img/IMG_4528.jpg" height="50%" width="50%">
+ Attach a Delrin plug each to two 9-Port Manifolds as shown.<br><img src="img/IMG_4529.jpg" height="50%" width="50%">
+ Cut seven 15cm pieces of PTFE Tubing for each 9-Port Manifold.<br><img src="img/IMG_4530.jpg" height="50%" width="50%">
+ Attach Plugs and Ferrules to all ports on both 9-Port Manifolds except the port opposite to the Delrin Plug.<br><img src="img/IMG_4532.jpg" height="50%" width="50%">
+ Attach both 9-Port Manifolds to the NO Port on the Time Point Valves:
    + The two PTFE Tubes from the Lysis Buffer Flush Valve Tee will be connected to the ports opposite to the Delrin Plug.
    + Connect NO Ports for the Valves 1 through 6 to the first 9-Port Manifold.
    + Connect the NO Ports for the Valves 7 through 12 to the second 9-Port Manifold.
    + You can lift up the Top of the Valve Holder for easier assembly.
    + Hand tighten all the way.<br><img src="img/IMG_4540.jpg" height="50%" width="50%">
+ Reassemble the Top of the Valve Holder. See below.
    + Make sure the the PTFE Tubing doesn't block the Top of the Valve Holder from attaching.
    
|Back View|Side View|Front View|
|-|-|-|
|<img src="img/IMG_4545.jpg">|<img src="img/IMG_4543.jpg">|<img src="img/IMG_4546.jpg">|

+ Cut three 8cm pieces of PTFE Tubing and assemble the Tee as shown.<br><img src="img/IMG_4547.jpg" height="50%" width="50%">
+ Without overtightening, connect the Front Port of the Tee to the Inlet Port for the Bead Channel flow Sensor.
    + Overtightening can break the glass inside the sensor and lead to erroneous flow values and blockages.
    + Do not tighten all the way. Stop once you feel some resistance.
    + Make sure you screw into the correct port of the sensor, by making sure the Flow Direction Arrow is in the correct orientation.<br><img src="img/IMG_4549.jpg" height="50%" width="50%">
+ Connect the two Side Ports of the Tee to the remaining Ports for the two 9 Port Manifolds.<br><img src="img/IMG_4554.jpg" height="50%" width="50%">
+ Cut a 11cm piece of PTFE Tubing and attach the Nuts and Ferrules.<br><img src="img/IMG_4555.jpg" height="50%" width="50%">
+ Attach one end of this Tubing to the back port for the Bead Bypass Valve as shown. <br><img src="img/IMG_4560.jpg" height="50%" width="50%">
+ Without overtightening, connect the other end of the Tubing to the Outlet Port for the Bead Channel flow Sensor.
    + Overtightening can break the glass inside the sensor and lead to erroneous flow values and blockages.
    + Do not tighten all the way. Stop once you feel some resistance.<br><img src="img/IMG_4561.jpg" height="50%" width="50%">
+ Next cut a 50.8cm piece of PTFE Tubing and attach a Nut and Ferrule to only one side of this Tubing.<br><img src="img/IMG_4562.jpg" height="50%" width="50%">
+ Attach the tubing to the NO Port on the Bead Bypass Valve using the Nut and Ferrule end.<br><img src="img/IMG_4563.jpg" height="50%" width="50%">
+ Push the other end of the Tubing into the Pipette Tip on the Linear Rail for the XYZ Robot.<br><img src="img/IMG_4564.jpg" height="50%" width="50%">
+ Secure the Tubing to the top of the XYZ Robot using labelling Tape.   
    + Execute the cell below to watch a video of how to thread the Tubing and secure it to the top of the XYZ Robot.<br><img src="img/IMG_4566.jpg" height="50%" width="50%">

In [2]:
%%HTML
<iframe width="800" height="600" src="https://www.youtube.com/embed/bTKQU7-jtuk?si=nSEFnJsx_sf2dPcn" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

+ [Test the XYZ Robot](instructions_for_setting_coordinates_for_xyz_robot.ipynb) to make sure the Tip of the Tubing is not colliding with any of the Tubes.
    + Pull the Tubing up or down if necessary.
+ Cut a 41.6cm piece of PTFE Tubing and attach a Nut and Ferrule to only one side of this Tubing.<br><img src="img/IMG_4571.jpg" height="50%" width="50%">
+ Attach this tubing to the NC Port of the Bead Bypass Valve.<br><img src="img/IMG_4572.jpg" height="50%" width="50%">
+ Tape the tubing to the left front leg of the Taller Monitor riser as shown.
    + Make sure the other end of the Tubing rests on the Microscope stage where it will be connected to our Microfluidic Chip.
    + Label the tubing "Bead" as shown, to indicate its from the Bead Channel.<br><img src="img/IMG_4574.jpg" height="50%" width="50%">
    
### Assembling the Cell Channel Tubing and Manifolds
+ Cut four 10cm pieces of PTFE Tubing and attach to the 5-Port manifold as shown, using Nuts and Ferrules.<br><img src="img/IMG_4576.jpg" height="50%" width="50%">
+ Connect these four ports on the 5-Port manifold to the NO Ports on the Cell Media Flush, Cell Sample Flush 1, Cell Sample Flush 2 and Cell Sample Flush 3 Valves.<br><img src="img/IMG_4578.jpg" height="50%" width="50%">
+ Cut a 22.2cm piece of PTFE Tubing and attach a Nut and Ferrule to only one side of the Tubing as shown.<br><img src="img/IMG_4581.jpg" height="50%" width="50%">
+ Connect this tubing to the remaining port on the 5-Port manifold as shown.
    + Secure the Tubing to the top of the Valve Holder using Labelling Tape.<br><img src="img/IMG_4580.jpg" height="50%" width="50%">
+ Remove the protective tape from the Cell Channel Flow Sensor outlet port.<br><img src="img/IMG_4583.jpg" height="50%" width="50%">
+ We will need the Red Ferrule included with your flow sensor for this next step.<br><img src="img/IMG_4584.jpg" height="50%" width="50%">
+ Cut a 19.5cm piece of PTFE Tubing, then:
    + Unscrew the Swagelok fitting your Cell Channel Flow Sensor Outlet Port and remove both metal Ferrule parts from it.
    + Place the Red Ferrule in the Orientation shown on the Tubing.
    + Also put a Standard Idex Nut and Ferrule on the other side of the Tubing.
    + Execute the cell below to watch a video of how to connect this tubing from your Cell Channel Flow Sensor outlet to your Cell Channel Bypass Valve.<br><img src="img/IMG_4585%20Copy.JPG" height="50%" width="50%">

In [5]:
%%HTML
<iframe width="800" height="600" src="https://www.youtube.com/embed/DyeBHlPMCSs?si=VZKUXsXdtJ2rkj2d" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

+ Cut a 41.6cm piece of PTFE Tubing and attach a Nut and Ferrule to only one side.<br><img src="img/IMG_4595.jpg" height="50%" width="50%">
+ Attach the Tubing to the NC Port on the Cell Bypass Valve.<br><img src="img/IMG_4596.jpg" height="50%" width="50%">
+ Route the Cable from behind and under the Cell and Bead Channel Flow Sensors till it reaches the microscope stage.
    + Label the Tubing as: Cell
    + We will connect this tubing directly to our Microfluidic Chip's Cell Inlet.
    + Secure the Tubing to the underside of the Cell and Bead Channel Flow sensors with Labelling Tape.
    
|Route from Behind|Tape under the Cell and Bead Channel Flow Sensors|Label: Cell|
|-|-|-|
|<img src="img/IMG_4599.jpg">|<img src="img/IMG_4598.jpg">|<img src="img/IMG_4597.jpg">|

+ Cut a 60cm piece of PTFE Tubing and attach a Nut and Ferrule to only one side.
    + Attach this Tubing to the NO Port of the Cell Bypass Valve.<br><img src="img/IMG_4600.jpg" height="50%" width="50%">
+ Insert the other end of this Tubing into the GL45 Bottle with the 3D printed Cap, under the Shorter Monitor Riser.
    + Insert a little bit only so the Tubing is near the bottom of the Bottle Cap as shown.<br><img src="img/IMG_4601.jpg" height="50%" width="50%">
+ Label the GL45 Bottle as "Bleach for Cells". 
    + Remember to put 100ml of Bleach in this bottle before you start your experiments.
    + The Tubing should not be immersed in the Bleach bottle.
    + Make sure the there is no bleach on the outside of the bottle which can accidently spill on the Cell and Oil channel flow sensors.
    + Bleach can corrode the Stainless steel inside the Flow Sensors. Be careful to avoid this possiblity.<br><img src="img/IMG_4602.jpg" height="50%" width="50%">
    
### Assembling the Oil Channel Tubing
+ Flip the Oil Channel Bypass Valve as shown, so the NC and NO Ports are facing towards you.<br><img src="img/IMG_4593.jpg" height="50%" width="50%">
+ Remove the protective Tape from the Oil Channel Flow Sensor Outlet.<br><img src="img/IMG_4603.jpg" height="50%" width="50%">
+ Cut a 15cm piece of PTFE Tubing:
    + Similar to described previously to for the Cell Channel, attach the Swagelok Fitting with the Red Ferrule.
    + Attach a Standard Idex Nut and Ferrule to the other side of the Tubing.<br><img src="img/IMG_4604.jpg" height="50%" width="50%">
+ Similar to described previously for the Cell Channel:
    + Attach the Swagelok side of the Tubing to the Oil Channel Sensor Outlet Port without overtightening.
    + Attach the other end to the back of the Oil Channel Bypass Valve.<br><img src="img/IMG_4605.jpg" height="50%" width="50%">
+ Cut a 41.6cm piece of PTFE Tubing and attach a Nut and Ferrule to only one side.
    + Attach this Tubing to the NC Port of the Oil Bypass Valve.<br><img src="img/IMG_4606.jpg" height="50%" width="50%">
+ Secure this tubing to the top of the Shorter Monitor riser in front of the Cell and Oil Channel Sensor Holder as shown.
    + Make sure the other end of the Tubing is on the Microscope stage and Labelled: Oil
    + We will insert this tubing directly into our Microfluidic chips's Oil inlet Port.<br><img src="img/IMG_4607.jpg" height="50%" width="50%">
    
### Assembling 50ml Flushing Reservoir for Cleaning Cell and Oil Channel Flow Sensors
+ Label the Pneumatic Tubing that connects to our Pressure Controller using a Push Connect fitting as: Pressure Controller

|<br>|<br>|
|-|-|
|<img src="img/IMG_4608.jpg">|<img src="img/IMG_4613.jpg">|

+ Now we will assemble our reservoir for cleaning the flow sensors with Acetone.You will need:
    + [1/4" NPT Female to 6mm Push Connect Adapter](https://www.amazon.com/dp/B0BBV2JSQP)
    + [1/4" NPT Male to Luer Adapter](https://www.mcmaster.com/51525K436/)<br><img src="img/IMG_4609.jpg" height="50%" width="50%">
+ Wrap PTFE Tape around the 1/4" NPT Male Thread.
    + Then using two wrenches to provide a counter-torque and attach the two pieces to each other.<br><img src="img/IMG_4610.jpg" height="50%" width="50%">
+ Next attach a [Luer to 3/32" Barb Adapter](https://www.mcmaster.com/51525K212/) as shown.

|<br>|<br>|
|-|-|
|<img src="img/IMG_4611.jpg">|<img src="img/IMG_4612.jpg">|

+ Put on your gloves, lab coat and safety glasses for the next steps.
+ We will need our [3-Port Reservoirs](instructions_for_device_assembly_and_setup.ipynb#3-Port-Reservoirs) and [1 1/4 OD X 3/4 ID X 1/16 Thickness Rubber Washers](https://www.amazon.com/Pack-Neoprene-Rubber-Washers-Thickness/dp/B017858EKC) for the next steps.
+ Insert a Rubber Washer inside a 3-Port Reservoir cap.<br><img src="img/IMG_4614.jpg" height="50%" width="50%">
+ Blow out any dust from the washer and from Inside the 3-Port Reservoir using the Compressed Air Gun.
    + Always remember to Turn the Pressure off after you are done using the Compressed Air Gun.
+ Screw the Reservoir onto a Acetone Resistant Polypropylene [50ml Falcon Tube](https://ecatalog.corning.com/life-sciences/b2c/US/en/Liquid-Handling/Tubes,-Liquid-Handling/Centrifuge-Tubes/Falcon%C2%AE-Conical-Centrifuge-Tubes/p/352098).<br><img src="img/IMG_4615.jpg" height="50%" width="50%">
+ Place three [Scienceware 50ml Tubes holders](https://www.fishersci.com/shop/products/bel-art-scienceware-conical-tube-holder/11999001) in the Sterlite Container as shown. <br><img src="img/IMG_4616.jpg" height="50%" width="50%">
+ Put the Reservoir in the 50ml tube holder. 
    + Then Hand-Tighten a [1/4-28 UNF to 3/32" Swivel Barb](https://www.idex-hs.com/store/product-detail/swivel_barb_adapter_for_use_with_soft_walled_tubing_polypropylene_each_for_3_32_2_4_mm_id_tubing/d-647) to the Reservoir cap as shown. <br><img src="img/IMG_4618.jpg" height="50%" width="50%">
+ Attach a [Delrin Plug](https://www.idex-hs.com/store/product-detail/plug_delrin_1_4_28_flat_bottom/p-309) to the second port.<br><img src="img/IMG_4619.jpg" height="50%" width="50%">
+ Next cut a 100cm segment of PTFE Tubing and 150cm segment of Tygon Tubing.
    + You will also need some [HPLC Grade Acetone](https://www.fishersci.com/shop/products/acetone-hplc-fisher-chemical-6/A949SK1).
    + Execute the cell below to watch a video of how to complete the assembly and testing of the Acetone Flush Reservoir.

In [15]:
%%HTML
<iframe width="800" height="600" src="https://www.youtube.com/embed/no5nJ0J_t4M?si=pvlBxW0VcPunXym3" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

+ Next we want to block any dust going into our Cell and Oil Channel flow sensors.
+ Two reservoirs are normally connected to the Inlet port of these Flow Sensors as part of the Protocol for operating the device.
+ Therefore we will use the Acetone Flush Reservoir and another Larger Reservoir for Blocking the Inlet ports for both sensors.
+ Execute the cell below to watch a video on how to connect the Acetone Flush Reservoir to the Cell Channel Flow Sensor inlet.

In [8]:
%%HTML
<iframe width="800" height="600" src="https://www.youtube.com/embed/6gWV8f0ALes?si=D787V8-x3DvRZbdp" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

### Assembling Second Larger Reservoir for Blocking Inlet Port of Flow Sensor
+ You will need a VICI 3-Port Reservoir Cap on a GL45 Bottle for assembling this reservoir.<br><img src="img/IMG_4646.jpg" height="50%" width="50%">
+ Cut a 100cm long Segment of PTFE Tubing.
    + Execute the cell below to watch a video of how to assemble and then connect this reservoir to our Oil Channel Flow Sensor Inlet.

In [2]:
%%HTML
<iframe width="800" height="600" src="https://www.youtube.com/embed/_8XUUv5e2AA?si=sB4Ej3hwO6kSTb1O" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

### Assembling Oil Channel Reservoirs and Tubing
+ When adding the O-Ring Gaskets to the CNC Machined 3-Port Caps make sure you use the Compressed Air Gun to remove any Dust.
+ Execute the cell below to watch a video on Dust Removal.

In [1]:
%%HTML
<iframe width="800" height="600" src="https://www.youtube.com/embed/iM0LEE1akXg?si=d__zJCKyAW_xedN2" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

+ From now onwards we will only assemble our CNC Machined 3 Port Reservoir Caps. **The common steps for all these reservoirs are:**
    + Use Compressed Air Gun to remove Dust from the Gasket.
    + For all these reservoirs we will add a O-Ring/Gasket to the inside portion of Reservoir Cap.
    + Use Compressed Air Gun to remove any remaining Dust from Gasket/Reservoir Cap.
    + Then we will screw in a 50ml Falcon Tube.
    + PTFE Tubing will be attached with a Nut and Ferrule.
+ Assemble the Oil Reservoir:
    + Attach a 85cm segment of PTFE Tubing.
    + Make sure the Tubing goes all the way to the bottom of the 50ml Tube.
    + Block one of the Ports with a Delrin Plug.
    + Attach a Swivel Barb to the last port.
    + Put the assembled Reservoir in the following 3D Printed Holder.
    + Label the Holder: Oil <br><img src="img/IMG_4654.jpg" height="50%" width="50%">
+ Place the other end of the PTFE Tubing attached to the reservoir on the Valve Holder near the Oil Flow Sensor Inlet.
    + This will be useful when have to attach it to the Flow Sensor during device operation.<br><img src="img/IMG_4655.jpg" height="50%" width="50%">
+ Assemble the Recovered Oil Reservoir:
    + Attach a 50cm segment of PTFE Tubing.
    + Make sure the Tubing goes all the way to the bottom of the 50ml Tube.
    + Block one of the Ports with a Delrin Plug.
    + Attach a Swivel Barb to the last port.
    + Put the assembled Reservoir in the following 3D Printed Holder.
    + Label the Holder: Recovered Oil 
    + Connect the other end of the Tubing to the NO Port of the Oil Bypass Valve.<br><img src="img/IMG_4660.jpg" height="50%" width="50%">
+ Attach a 40cm or Longer Segment of Tygon Tubing to the Swivel Barb on the Recovered Oil Reservoir.
    + Make sure the other end of the Tygon Tubing is in the Secondary Containers below the Shorter Monitor Riser.
    + The Purpose of this Tygon Tubing is to expose the Recovered Oil Reservoir to Atmospheric Pressure.
    + Moreover, we don't want Dust to enter the Reservoir. Therefore by using a Segment of Tygon tubing we prevent Dust from entering the Reservoir while also keeping it at Atmospheric Pressure.
    + Exact Tygon Tubing length doesn't matter a lot.<br><img src="img/IMG_4661.jpg" height="50%" width="50%">
    
+ Next attach a [3/32" Barb to Luer Adapter from Idex](https://www.idex-hs.com/store/product-detail/barbed_to_male_luer_lock_adapter_for_use_with_soft_walled_tubing_polypropylene_for_use_with_3_32_2_4_mm_id_tubing_each/p-851) to the Double Luer Filter included with the Pressure Controller as shown.

|3/32 inch Barb to Luer Adapter and Filter Included with Pressure Controller| After Joining|
|-|-|
|<img src="img/IMG_4665.jpg">|<img src="img/IMG_4666.jpg">|

+ Remove the Luer Plug from Channel 2 on the Pressure Controller.
    + Then attach the Assembled Luer Barb+Filter.

|Remove the Luer Plug from Channel 2| Connect Assembled Luer+Filter|
|-|-|
|<img src="img/IMG_4664.jpg">|<img src="img/IMG_4667.jpg">|

+ Attach Tygon Tubing to the Luer Barb on Channel 2.
    + Tygon Tubing should be long enough to reach the Oil Reservoir. 
    + Its good to leave some slack in the Tygon Tubing.
    + Exact tubing length doesn't matter a lot.
    
|Attach Tygon Tubing| Connect the other end to Oil Reservoir Swivel Barb.|
|-|-|
|<img src="img/IMG_4668.jpg">|<img src="img/IMG_4669.jpg">|

### Assembling Cell Channel Reservoirs and Tubing

+ Attach the Gasket and then Label three Reservoir Caps: 1, 2 and 3.
    + These will be our Cell Sample Reservoirs 1, 2 and 3. <br><img src="img/IMG_4670.jpg" height="50%" width="50%">
+ Attach the following Fittings for two of the Ports of each Reservoir:
    + For Reservoir 1: Two Swivel Barbs.
    + For Reservoir 2: One Delrin Plug and One Swivel Barb.
    + For Reservoir 3: One Delrin Plug and One Swivel Barb.<br><img src="img/IMG_4671.jpg" height="50%" width="50%">
+ Assemble three 50ml Tube Holders from Bel-Art Scienceware.
    + These can be used later on the Wet lab portion of your Bench.
+ Cut three 74cm Segments of PTFE Tubing for each Reservoir:
    + Attach the segments to the third remaining port for each reservoir.
    + Make sure the Tubing goes all the way till the bottom of the 50ml Falcon Tube.
    + Attach a Ferrule and Nut on the other side of the Tubing.<br><img src="img/IMG_4672.jpg" height="75%" width="75%">
+ Put the Reservoirs in the clamps in your water bath as shown.
    + Reservoir 1 should be in the Top Right Clamp next to the Magnetic Stirrer.
        + Adjust the position of the Magnetic Stirrer if necessary to make sure Central axes of Reservoir 1 and the Magnetic Stirrer as aligned along the same plane.
    + Reservoir 2 and 3 Positions should also match the positions shown in the picture.<br><img src="img/IMG_4675.jpg" height="75%" width="75%">
+ Attach the Reservoirs to their respective Valves.
    + Attach the Tubing from Cell Sample Reservoir 1 to the NC Port on Cell Sample Flush Valve 1
    + Attach the Tubing from Cell Sample Reservoir 2 to the NC Port on Cell Sample Flush Valve 2
    + Attach the Tubing from Cell Sample Reservoir 3 to the NC Port on Cell Sample Flush Valve 3
    + Adjust the position of the Water Bath, Sterlite Container, and Magnetic Stirrer if necessary.
    + The Tubing should reach the Valves comfortably with a little bit of slack left in the Tubing.
    + Make sure the Central axes of Reservoir 1 and the Magnetic Stirrer as aligned along the same plane.
    
|Side View of the Valves| Top View of the Valves|
|-|-|
|<img src="img/IMG_4676.jpg">|<img src="img/IMG_4677.jpg">|

+ Attach the Gasket and then Label two Reservoir Caps :F
    + These will be our Cell Media Flush Reservoirs. <br><img src="img/IMG_4678.jpg" height="50%" width="50%">
+ Assemble the Reservoir and then attach two Swivel Barbs each to both. <br><img src="img/IMG_4679.jpg" height="50%" width="50%">
+ Assemble a Tee as follows:
    + Attach 70cm Segments of PTFE Tubing to both side ports.
    + Attach a 45cm segment of PTFE Tubing to the Front Port.<br><img src="img/IMG_4680.jpg" height="50%" width="50%">
+ Attach the Front Port Tubing to the NC Port of the Cell Media Flush Valve.<br><img src="img/IMG_4681.jpg" height="50%" width="50%">
+ Keep the Tee inside the Sterlite Container as shown.<br><img src="img/IMG_4682.jpg" height="50%" width="50%">
+ Attach the Tubing from the Side Ports of the Tee to the Cell Media Flush Reservoirs.
    + Place the Cell Media Flush Reservoirs in the Clamps inside the Water Bath as shown.
    + Make sure the Tubing goes all the way till the bottom of the 50ml Falcon Tube.<br><img src="img/IMG_4683.jpg" height="50%" width="50%">
+ Attach Swivel Barbs to both the NC and NO Ports on the Cell Gas Vent Valve.<br><img src="img/IMG_4684.jpg" height="50%" width="50%">
+ Attach a 50cm Segment of Tygon Tubing to the NC Port Swivel Barb.
    + Make sure the other end of the Tubing is in the Secondary Container under the Lower Monitor Riser.<br><img src="img/IMG_4685.jpg" height="50%" width="50%">
+ Cut a 50cm Segment of Tygon Tubing and attach it to the Swivel Barb on the NO Port of the Gas Vent Valve. 
    + Hang the other end of the Tubing inside the Sterlite Container.

|50cm Segment of Tygon Tubing to NC Port of Gas Vent Valve| Put the other side inside the Sterlite Container|
|-|-|
|<img src="img/IMG_4689.jpg">|<img src="img/IMG_4690.jpg">|

+ Next we will make a Gas Manifold using Tygon Tubing and [Luer Tees](https://www.mcmaster.com/51525K443/) ordered previously.<br><img src="img/IMG_4687.jpg" height="50%" width="50%">
+ Attach [3/32" Barb to Luer Adapters from Idex](https://www.idex-hs.com/store/product-detail/barbed_to_male_luer_lock_adapter_for_use_with_soft_walled_tubing_polypropylene_for_use_with_3_32_2_4_mm_id_tubing_each/p-851) to the Luer Tees as shown.<br><img src="img/IMG_4688.jpg" height="50%" width="50%">
+ Join the two Luer Tees together with a 17cm segment of Tygon Tubing as shown.<br><img src="img/IMG_4691.jpg" height="50%" width="50%">
+ Connect three 50cm segments of Tygon Tubing:
    + Two to the side ports of the lower Luer Tee
    + One to the left side port of the upper tee as shown.<br><img src="img/IMG_4692.jpg" height="50%" width="50%">
+ Connect the Tygon Tubing connected to the NO Port of the Gas Vent Valve to the Remaining Front Port of the Upper Luer Tee.<br><img src="img/IMG_4693.jpg" height="50%" width="50%">
+ Connect the Tygon Tubing connected to the left upper Luer tee to one of the Swivel Barbs on Cell Sample Reservoir 1.<br><img src="img/IMG_4694.jpg" height="50%" width="50%">
+ Connect the remaining Ports on the Lower Luer tee to one Swivel Barb each on both Cell Media Flush Reservoirs.<br><img src="img/IMG_4695.jpg" height="50%" width="50%">
+ We have now setup 5%CO<sub>2</sub> venting for our Cell Sample Reservoir 1, and Cell Media Flush Reservoirs.
    + Therefore if your experiment requires a 5%CO<sub>2</sub> environment for your Cells, make sure to only use Cell Sample Reservoir 1.
+ We will now create a Gas Manifold for connecting our Pressure Controller to our Cell Channel Reservoirs. Assemble four Luer tees as shown.<br><img src="img/IMG_4696.jpg" height="50%" width="50%">
+ Connect the Luer Tees with 5cm Tygon Tubing Segments as shown.<br><img src="img/IMG_4697%20(1).jpg" height="50%" width="50%">
+ We will now connect this Gas Manifold to our Reservoirs. The segment lengths and which Reservoirs these segments will connect to, are described in the image below.
    + 70cm segments for Reservoir 1 and 3.
    + 50cm segments for Reservoir 2, and the Flush Reservoirs.<br><img src="img/IMG_4697.jpg" height="50%" width="50%">
+ Attach the Tygon Segments.<br><img src="img/IMG_4698.jpg" height="50%" width="50%">
+ Connect the Gas Manifold using the Tygon Tubing segments to the Swivel Barbs on those specific Reservoirs.<br><img src="img/IMG_4699.jpg" height="75%" width="75%">
+ Next attach a [3/32" Barb to Luer Adapter from Idex](https://www.idex-hs.com/store/product-detail/barbed_to_male_luer_lock_adapter_for_use_with_soft_walled_tubing_polypropylene_for_use_with_3_32_2_4_mm_id_tubing_each/p-851) to the Double Luer Filter included with the Pressure Controller as shown.<br><img src="img/IMG_4700.jpg" height="50%" width="50%">
+ Connect this Assembled Filter to the Luer Barb 1 on the Pressure Controller. Channel 1 is for the Cell Channel.<br><img src="img/IMG_4701.jpg" height="50%" width="50%">
+ Using Tygon Tubing Connect Channel 1 of the Pressure Controller to the Gas Manifold for the Cell Channel Reservoirs.
    + There should be only one remaining position on the Gas manifold. Also descibed above.
    + Exact length of Tygon Tubing doesn't matter too much.
    + Leave some slack in the Tubing.
    + Route the Tubing from behind the Water bath to avoid tangles.
<br><img src="img/IMG_4706.jpg" height="75%" width="75%">

### Assembling Bead Channel Reservoirs and Tubing
+ We will now assemble the Reservoir caps for the Bead Channel.
+ After adding the Gasket and removing dust from 14 caps, label the caps as follows:
    + Label 12 Caps, 1 through 12. These will be for our Bead Time Point Reservoirs. 
        + Each cap will be connected to its corresponding Time Point Valve with the name Number Label.
    + Label 2 Caps as "L" for Lysis Buffer. These will be our Bead Lysis Buffer Reservoirs. 
        + These reservoirs will be connected to the Bead Lysis Buffer Flush Valve.<br><img src="img/IMG_4711.jpg" height="50%" width="50%">
+ For all 14 reservoirs:
    + Block one port with a Delrin Plug.
    + Attach a Swivel Barb to another port.<br><img src="img/IMG_4712.jpg" height="50%" width="50%">
+ Next, for all 12 Time point reservoirs:
    + Attach a 60cm segment of PTFE Tubing to the remaining port.
    + Attach a Nut and Ferrule to the other end for connecting to the Time point Valves.
    + **Leave a slight gap between the tip of the PTFE Tubing and the Base of the 50ml Tube.**
        + This small gap is very important and helps avoid Bead Blockages inside the PTFE Tubing.
        + This also allows free movement of the PTFE Tubing when the Vortices are turned on, allowing for better mixing.
        + Leaving a large gap can lead to a high dead volume and bubbles of air injected into the Microfluidic chip.
        + These bubbles can completely ruin an injection and potentially also lead to blockages in the microfluidic chip.
        + Execute the cell below to watch a video about this gap and how to adjust it manually.
        
|Attach a 60cm PTFE tubing Segment.| Leave a small gap from the base. About 1mm.|
|-|-|
|<img src="img/IMG_4720.jpg">|<img src="img/IMG_4719.jpg">|

In [3]:
%%HTML
<iframe width="800" height="600" src="https://www.youtube.com/embed/j0GCYF-Xlps?si=hshab3uxXW2S6y5g" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

+ Insert the Bead Time Point Reservoirs 1 through 6 into Vortex 1 as shown.<br><img src="img/IMG_4723.jpg" height="50%" width="50%">
+ Connect the tubing from each Reservoir to the NC Port of its corresponding Time point Valve. 
    + The Reservoirs and Time point valves being connected should have the same Number Label. 
    + For example, Reservoir 1 connects to Time point Valve 1.<br><img src="img/IMG_4725.jpg" height="50%" width="50%">
+ Insert the Bead Time Point Reservoirs 7 through 12 into Vortex 2 as shown.<br><img src="img/IMG_4726.jpg" height="50%" width="50%">
+ Connect the tubing from each Reservoir to the NC Port of its corresponding Time point Valve. 
    + The Reservoirs and Time point valves being connected should have the same Number Label. 
    + For example, Reservoir 7 connects to Time point Valve 7.<br><img src="img/IMG_4727.jpg" height="50%" width="50%">
+ Next we need to define which Bead Time Point Reservoirs are in each Vortex.
+ Populate the lists below with the Number Labels in the First and Second Vortices.
    + We put 1 through 6 in Vortex 1 so the list ```bead_vortex_reservoirs``` will contain the numbers 1 through 6.
    + We put 7 through 12 in Vortex 2 so the list ```bead_vortex2_reservoirs``` will contain the numbers 7 through 12.
    + You should change these values if you made modifications or a decided to use different number labels in the vortices.
+ Execute the cell below to create the ```config/number_label_in_each_vortex.py``` file with this information.

In [6]:
%%write_and_run config/number_label_in_each_vortex.py
bead_vortex_reservoirs=[1,2,3,4,5,6]#Number Labels in Vortex 1
bead_vortex2_reservoirs=[7,8,9,10,11,12]#Number Labels in Vortex 2

+ For the two Lysis Buffer Flush Reservoirs:
    + Attach 60cm segments of PTFE Tubing to the remaining ports.
    + Make sure the tubing goes all the way down to the base of the 50ml Tubes.
    + Attach the other ends of the Tubing from both reservoirs to the side ports of a Tee.<br><img src="img/IMG_4721.jpg" height="75%" width="75%">
+ Assemble a Luer Tee and attach 30cm long segments of Tygon Tubing to both the side ports.<br><img src="img/IMG_4729.jpg" height="50%" width="50%">
+ Attach the Luer Tee to Swivel Barbs on Both the Lysis Bufffer Flush Reservoirs.<br><img src="img/IMG_4730.jpg" height="50%" width="50%">
+ Next we will make a Gas Manifold for connecting our Bead Channel Reservoirs to the Pressure Controller.
    + Assemble 12 Luer Tees as shown.<br><img src="img/IMG_4731.jpg" height="50%" width="50%">
+ Connect the 12 Luer Tees with 2cm segments of Tygon Tubing.<br><img src="img/IMG_4734.jpg" height="75%" width="75%">
+ We will now connect this Gas Manifold to our Bead Time Point Reservoirs. The segment lengths and which Reservoirs these segments will connect to, are described in the image below.
    + 110cm Tygon Tubing segments for Reservoir 1 through 12.
    + 30cm segment for connecting to the Front Port on the Luer Tee for the Lysis Buffer Flush Reservoirs.<br><img src="img/IMG_4734%20(1).jpg">
+ Put three [23 inch Lab Stands](https://www.amazon.com/dp/B0D75SPCB7) next to the Vortices as shown.<br><img src="img/IMG_4735.jpg" height="50%" width="50%">
+ Secure the Lab Stands to each other using four [3-Finger Clamps](https://www.amazon.com/dp/B00AWS3QV0).
    + Put two clamps below the level of the Vortex Heads.
    + Put two clamps above the Valve Holder.
    + The purpose of doing this is to reduce vibrations when the Vortices start shaking.<br><img src="img/IMG_4737.jpg" height="75%" width="75%">
+ Put two [3-Finger clamps](https://www.amazon.com/dp/B00AWS3QV0) above the Vortex Heads to hold in place the Tygon Tubing that will attach to the Reservoirs.

|Front View|Side View|
|-|-|
|<img src="img/IMG_4739.jpg">|<img src="img/IMG_4740.jpg">|

+ Put the Lysis Buffer Flush Reservoirs in an [Ice Box](https://www.heathrowscientific.com/transformer-cube-tube-rack/) next to Vortex 2 and the XYZ Robot as shown.<br><img src="img/IMG_4741.jpg" height="50%" width="50%">
+ Secure the Ice Box in place on your bench by using Labelling Tape on the base as shown.<br><img src="img/IMG_4742.jpg" height="50%" width="50%">
+ Put the Gas Manifold next to the Lysis Buffer Flush Reservoirs and Lab Stands.<br><img src="img/IMG_4743.jpg" height="50%" width="50%">
+ Connect the previously described Assembled Filter to Luer Barb 3 on the Pressure Controller. Channel 3 is for the Bead Channel.<br><img src="img/IMG_4744.jpg" height="50%" width="50%">
+ Using Tygon Tubing Connect Channel 3 of the Pressure Controller to the Gas Manifold for the Bead Channel Reservoirs.
    + Exact length of Tygon Tubing doesn't matter too much.
    + Leave some slack in the Tubing.
    + Route the Tubing from behind the ChronoSeq device to avoid tangles.

|Connect to Channel 3|Route from behind the ChronoSeq device.|
|-|-|
|<img src="img/IMG_4747.jpg">|<img src="img/IMG_4746.jpg">|

+ Using a 60cm Segment of PTFE Tubing:
    + Connect the Front Port of the Tee for the Lysis Buffer Flush Reservoirs to the NC Port of the Lysis Buffer Flush Valve.

|Lysis Buffer Flush Reservoir Tee Front Port |To NC Port of Lysis Buffer Flush Valve|
|-|-|
|<img src="img/IMG_4748.jpg">|<img src="img/IMG_4765.jpg">|

+ Using a 30cm segment of Tygon Tubing connect the Front Port of the Luer Tee for the Lysis Buffer Flush Reservoirs to the Gas Manifold.<br><img src="img/IMG_4749.jpg" height="50%" width="50%">
+ Using 110cm segments of Tygon Tubing connect the Swivel Barbs on Reservoirs 1 through 6 to the Gas Manifold.
    + Route the Tygon Tubing through the 3-Finger Clamp above Vortex 1.
    + Tighten the Clamp to Hold the Tygon Tubing in place.
    + Leave some slack from the Clamp till the Swivel Barb to allow free movement of the Reservoirs during mixing.
    + Use Labelling Tape to Secure the Tygon Tubing to the Lab Stand as shown.<br><img src="img/IMG_4751.jpg" height="75%" width="75%">
+ Using 110cm segments of Tygon Tubing connect the Swivel Barbs on Reservoirs 7 through 12 to the Gas Manifold.
    + Route the Tygon Tubing through the 3-Finger Clamp above Vortex 2.
    + Tighten the Clamp to Hold the Tygon Tubing in place.
    + Leave some slack from the Clamp till the Swivel Barb to allow free movement of the Reservoirs during mixing.
    + Use Labelling Tape to Secure the Tygon Tubing to the Lab Stand as shown.<br><img src="img/IMG_4752.jpg" height="75%" width="75%">
+ Label a 2 Port Union "Outlet Flush" and put it on the Microscope stage as shown.<br><img src="img/IMG_4753.jpg" height="50%" width="50%">
+ Secure the Union to the Microscope stage using Labelling Tape with the message: "For Outlet Flush"<br><img src="img/IMG_4754.jpg" height="50%" width="50%">
+ Label a 2 Port Union "Cell" and put it on the Microscope stage next to the other Union as shown.<br><img src="img/IMG_4755.jpg" height="50%" width="50%">
+ Secure the Union to the Microscope stage using Labelling Tape with the message: "For Bulk RNA-Seq"<br><img src="img/IMG_4756.jpg" height="50%" width="50%">
+ Cut a 52.7cm segment of PTFE Tubing:
    + Insert one end into the Linear Rail with the Pipette Tip for the XY Robot.
    + Put the other end on the Microscope stage.
    + Leave enough slack for the XY Robot to easily move the Tubing between different positions.
    + Secure the tubing to the Microscope stage with Labelling Tape.
    + Write the message on the Labelling Tape: Outlet Tubing XY Robot.
    + We will connect this Tubing directly to our Microfluidic Chip's outlet.

|Attach to Linear Rail on XY Robot. Leave enough slack. |Tape to Microscope stage.|
|-|-|
|<img src="img/IMG_4757.jpg">|<img src="img/IMG_4759.jpg">|

+ [Test the XYZ and XY Robots](instructions_for_setting_coordinates_for_xyz_robot.ipynb) and adjust the Outlet Tubing position if necessary.
+ You will have the opportunity to Test the Vortices when you are running the ChronoSeq device, so you can skip this part for now if you want.
    + [Test the Vortices](instructions_for_device_assembly_and_setup.ipynb#Setting-up-the-Vortices-and-their-Relay-Controller) to make sure they can shake freely for evenly suspending the Beads when turned on.
    + Manually adjust the speed to make sure the Vortex speed is sufficient.
    + High speeds can destabilize the Vortex and cause Bead shearing.
    + Use the minimum speed that works best for evenly suspending the Beads.
    + Create more slack in the Tygon Tubing if it is impeding free rotation of the Vortices.
    + Tape the Speed Dial on both vortices after you have set the speed to prevent accidental changes in rotation speed.

|Tape Speed Dial on Vortex 1 after setting best Minimum speed |Tape Speed Dial on Vortex 2 after setting best Minimum speed|
|-|-|
|<img src="img/IMG_4763.jpg">|<img src="img/IMG_4762.jpg">| 

### Assembly Completion notes and next steps
+ You need to determine the Scaling Factor again for your particular setup to get accurate flow rates.
    + The Bead Channel Flow Sensor is a Thermal flow based sensor and needs to be calibrated when the Lysis Buffer recipe is changed.
    + The Ficoll concentration has the strongest effect on the Scaling Factor.
    + The empirically determined Scaling Factor for our Lysis Buffer recipe will be stored in in ```config/beadFlowSensorScalingFactor.py```.
    + To measure the Scaling factor see the [section for Calibrating the Bead Flow Sensor](protocol_and_software_for_running_chronoseq_device.ipynb#Calibrate-the-Bead-Channel-Flow-Sensor) in the notebook for running the ChronoSeq Device.
+ The ChronoSeq device is now assembled and you can go ahead and [run the device](protocol_and_software_for_running_chronoseq_device.ipynb).
    + We recommend doing a dummy run with no beads and cells to make sure everything is working correctly, before moving on to the real thing.